In [2]:
import random
import json
import numpy as np
import pickle
import tensorflow as tf
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [14]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('./intents.json').read())
intents

{'intents': [{'tag': 'greetings',
   'patterns': ['hello', 'hai', 'hi', 'hey', 'halo', 'good day'],
   'responses': ['Hello', 'Hi', 'What can i do for you?']},
  {'tag': 'goodbye',
   'patterns': ['bye', 'cya', 'see you later', 'daa'],
   'responses': ['Ok bye', 'Bye', 'Nice to talk with you', 'See ya']},
  {'tag': 'age',
   'patterns': ['how old are you',
    'how old r u',
    'how old',
    'your age?',
    'age?'],
   'responses': ['I have created since 2023']},
  {'tag': 'name',
   'patterns': ['what is your name',
    'who are you',
    'your name?',
    'do you have a name',
    'what should i call you',
    'tell me your name'],
   'responses': ['You can call me bataraprabu',
    "I'm bataraprabu",
    'My name is bataraprabu']}]}

In [4]:
intents_df = pd.read_json('./intents.json')

In [5]:
intents_df.head()

,intents
0,"{'tag': 'greetings', 'patterns': ['hello', 'ha..."
1,"{'tag': 'goodbye', 'patterns': ['bye', 'cya', ..."
2,"{'tag': 'age', 'patterns': ['how old are you',..."
3,"{'tag': 'name', 'patterns': ['what is your nam..."


In [62]:
# Flatten data
df_nested = pd.json_normalize(intents, record_path =['intents'])
df_nested = df_nested[['tag', 'patterns']]
df_nested = df_nested.explode(['patterns'])
# df_nested = pd.DataFrame(df_nested)
df_nested

,tag,patterns
0,greetings,hello
0,greetings,hai
0,greetings,hi
0,greetings,hey
0,greetings,halo
0,greetings,good day
1,goodbye,bye
1,goodbye,cya
1,goodbye,see you later
1,goodbye,daa


In [65]:
df_nested[['greetings', 'goodbye', 'name']] = pd.get_dummies(df_nested['tag'], drop_first=True)
df_nested

,tag,patterns,greetings,goodbye,name
0,greetings,hello,0,1,0
0,greetings,hai,0,1,0
0,greetings,hi,0,1,0
0,greetings,hey,0,1,0
0,greetings,halo,0,1,0
0,greetings,good day,0,1,0
1,goodbye,bye,1,0,0
1,goodbye,cya,1,0,0
1,goodbye,see you later,1,0,0
1,goodbye,daa,1,0,0


In [3]:
words=[]
classes=[]
documents=[]
ignore_letters=['?', '!', '.', ',']

In [4]:
tokenizer = word_tokenize

In [5]:
'intents.json-> intents-> {tag, patterns, responses}' "ket: -> berarti berisi"

'intents.json-> intents-> {tag, patterns, responses}ket: -> berarti berisi'

In [6]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = tokenizer(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
        

In [7]:
print(documents)

[(['hello'], 'greetings'), (['hai'], 'greetings'), (['hi'], 'greetings'), (['hey'], 'greetings'), (['halo'], 'greetings'), (['good', 'day'], 'greetings'), (['bye'], 'goodbye'), (['cya'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['daa'], 'goodbye'), (['how', 'old', 'are', 'you'], 'age'), (['how', 'old', 'r', 'u'], 'age'), (['how', 'old'], 'age'), (['your', 'age', '?'], 'age'), (['age', '?'], 'age'), (['what', 'is', 'your', 'name'], 'name'), (['who', 'are', 'you'], 'name'), (['your', 'name', '?'], 'name'), (['do', 'you', 'have', 'a', 'name'], 'name'), (['what', 'should', 'i', 'call', 'you'], 'name'), (['tell', 'me', 'your', 'name'], 'name')]


In [8]:
print(words)

['hello', 'hai', 'hi', 'hey', 'halo', 'good', 'day', 'bye', 'cya', 'see', 'you', 'later', 'daa', 'how', 'old', 'are', 'you', 'how', 'old', 'r', 'u', 'how', 'old', 'your', 'age', '?', 'age', '?', 'what', 'is', 'your', 'name', 'who', 'are', 'you', 'your', 'name', '?', 'do', 'you', 'have', 'a', 'name', 'what', 'should', 'i', 'call', 'you', 'tell', 'me', 'your', 'name']


In [9]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

In [10]:
print(words)

['a', 'age', 'are', 'bye', 'call', 'cya', 'daa', 'day', 'do', 'good', 'hai', 'halo', 'have', 'hello', 'hey', 'hi', 'how', 'i', 'is', 'later', 'me', 'name', 'old', 'r', 'see', 'should', 'tell', 'u', 'what', 'who', 'you', 'your']


In [11]:
classes = sorted(set(classes))

In [12]:
pickle.dump(words,open('words.pkl', 'wb'))
pickle.dump(classes,open('classes.pkl', 'wb'))

In [13]:
training = []
output_empty = [0]*len(classes)
print(list(output_empty))

[0, 0, 0, 0]


# one hot encoding

In [14]:
classes.index('greetings')

2

In [15]:
for doc in documents:
    bag = []
    word_pattern = doc[0]
    word_pattern = [lemmatizer.lemmatize(word.lower()) for word in word_pattern]
    for word in words:
        bag.append(1) if word in word_pattern else bag.append(0) #one hot encoding
    
    label_class = list(output_empty)
    label_class[classes.index(doc[1])]=1
    training.append([bag, label_class])
        

In [16]:
random.shuffle(training)
training = np.array(training)

C:\Users\asus\AppData\Local\Temp\ipykernel_9744\2195589081.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [17]:
X_train = list(training[:, 0])
y_train = list(training[:, 1])

In [18]:
print(len(X_train[0]), len(X_train[6]))

32 32


In [67]:
type(np.array(X_train))

numpy.ndarray

In [93]:
model = Sequential()
model.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation= 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))

In [94]:
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 128)               4224      
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 4)                 260       
                                                                 
Total params: 12,740
Trainable params: 12,740
Non-trainable params: 0
_________________________________________________________________
None


In [95]:
sgd= SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [96]:
model.fit(np.array(X_train), np.array(y_train), epochs=200, verbose=1)

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 1.3938 - accuracy: 0.1905
Epoch 2/200
1/1 [==============================] - 0s 42ms/step - loss: 1.4808 - accuracy: 0.2381
Epoch 3/200
1/1 [==============================] - 0s 32ms/step - loss: 1.4163 - accuracy: 0.1429
Epoch 4/200
1/1 [==============================] - 0s 38ms/step - loss: 1.3614 - accuracy: 0.3333
Epoch 5/200
1/1 [==============================] - 0s 39ms/step - loss: 1.4011 - accuracy: 0.4286
Epoch 6/200
1/1 [==============================] - 0s 28ms/step - loss: 1.4018 - accuracy: 0.2381
Epoch 7/200
1/1 [==============================] - 0s 34ms/step - loss: 1.3707 - accuracy: 0.3333
Epoch 8/200
1/1 [==============================] - 0s 34ms/step - loss: 1.3498 - accuracy: 0.3810
Epoch 9/200
1/1 [==============================] - 0s 37ms/step - loss: 1.5040 - accuracy: 0.2381
Epoch 10/200
1/1 [==============================] - 0s 36ms/step - loss: 1.3261 - accuracy: 0.3810
Epoch 11/200
1/1 [===

1/1 [==============================] - 0s 17ms/step - loss: 0.1204 - accuracy: 0.9524
Epoch 166/200
1/1 [==============================] - 0s 17ms/step - loss: 0.1318 - accuracy: 1.0000
Epoch 167/200
1/1 [==============================] - 0s 17ms/step - loss: 0.0890 - accuracy: 1.0000
Epoch 168/200
1/1 [==============================] - 0s 17ms/step - loss: 0.1518 - accuracy: 1.0000
Epoch 169/200
1/1 [==============================] - 0s 17ms/step - loss: 0.1227 - accuracy: 1.0000
Epoch 170/200
1/1 [==============================] - 0s 17ms/step - loss: 0.0898 - accuracy: 1.0000
Epoch 171/200
1/1 [==============================] - 0s 17ms/step - loss: 0.0750 - accuracy: 1.0000
Epoch 172/200
1/1 [==============================] - 0s 17ms/step - loss: 0.1617 - accuracy: 0.9524
Epoch 173/200
1/1 [==============================] - 0s 17ms/step - loss: 0.0583 - accuracy: 1.0000
Epoch 174/200
1/1 [==============================] - 0s 17ms/step - loss: 0.0720 - accuracy: 1.0000
Epoch 175/200


In [91]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

In [37]:
def cleansing(text):
    word = word_tokenize(text)
    word = [lemmatizer.lemmatize(w) for w in word]
    return word  

In [106]:
def bagofword(text):
    word = cleansing(text)
    bag = [0]*len(words)
    for w in word:
        for i, t in enumerate(words):
            if t == w:
                bag[i] = 1
#     return print(type(tf.convert_to_tensor(bag, dtype=tf.float32)))
#     return print(type(np.array(bag)))
    return np.array(bag).reshape(1,32)
    
    

In [112]:
def predict(t):
    bow = bagofword(t)
    res = model.predict(np.array(bow))[0]
    error_treshold = 0.25
    result = [[i,r] for i,r in enumerate(res) if r > error_treshold]
    
    result.sort(key=lambda x: x[1],reverse=True)
    return_list =[]
    for r in result:
        return_list.append({'intents': classes[r[0]], 'probability' : str(r[1])})
    return return_list

In [113]:
def get_response(intent_list, intent_json):
    tag = intent_list[0]['intents']
    list_of_intent = intent_json['intents']
    for i in list_of_intent:
        if i['tag'] == tag:
            result=random.choice(i['responses'])
            break
    return result

In [114]:
print('Bot is Running')

Bot is Running


In [ ]:
while True:
    message=input("")
    ints = predict(message)
    res= get_response(ints, intents)
    print(res)

halo
1/1 [==============================] - 0s 31ms/step
Hello
name
1/1 [==============================] - 0s 78ms/step
My name is bataraprabu
how
1/1 [==============================] - 0s 78ms/step
I have created since 2023
hoe are you
1/1 [==============================] - 0s 78ms/step
You can call me bataraprabu
how old are you
1/1 [==============================] - 0s 78ms/step
I have created since 2023
how old are you
1/1 [==============================] - 0s 78ms/step
I have created since 2023
your name
1/1 [==============================] - 0s 78ms/step
You can call me bataraprabu


In [108]:
h='halo'
bagofword(h).shape

(1, 32)

In [103]:
np.array(X_train).shape

(21, 32)